<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### MVP Theory over Time

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Out-of-Sample Testing

Topics of interest include:

* mean-variance portfolio theory
 * in-sample approach
 * out-of-sample testing

## Real Financial Data

**_Historical end-of-day financial time series data._**

See AI in Finance (ch. 04) and `http://hilpisch.com/aiif_eikon_eod_data.csv`.

## Imports and Data

In [ ]:
import numpy as np
import pandas as pd
from pylab import plt, mpl
from scipy.optimize import minimize
plt.style.use('seaborn-v0_8')
np.set_printoptions(precision=5, suppress=True,
                   formatter={'float': lambda x: f'{x:6.3f}'})

In [ ]:
url = 'http://hilpisch.com/aiif_eikon_eod_data.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True).dropna()

In [ ]:
raw.info()

In [ ]:
symbols = ['AAPL.O', 'MSFT.O', 'INTC.O', 'AMZN.O', 'GLD', 'EUR=']

In [ ]:
# symbols = ['SPY', 'GLD', 'EUR=']

In [ ]:
(raw[symbols[:]] / raw[symbols[:]].iloc[0]).plot(figsize=(10, 6));

## Mean-Variance Portfolio Theory

In [ ]:
rets = np.log(raw[symbols] / raw[symbols].shift(1)).dropna()

In [ ]:
weights = len(rets.columns) * [1 / len(rets.columns)]
weights

In [ ]:
def port_return(rets, weights):
    return np.dot(rets.mean(), weights) * 252  # annualized

In [ ]:
port_return(rets, weights)

In [ ]:
def port_volatility(rets, weights):
    return np.dot(weights, np.dot(rets.cov() * 252 , weights)) ** 0.5  # annualized

In [ ]:
port_volatility(rets, weights)

In [ ]:
def port_sharpe(rets, weights):
    return port_return(rets, weights) / port_volatility(rets, weights)

In [ ]:
port_sharpe(rets, weights)

In [ ]:
w = np.random.random((1000, len(symbols)))
w = (w.T / w.sum(axis=1)).T

In [ ]:
w[:5]

In [ ]:
w[:5].sum(axis=1)

In [ ]:
pvr = [(port_volatility(rets[symbols], weights),
        port_return(rets[symbols], weights))
       for weights in w]
pvr = np.array(pvr)

In [ ]:
psr = pvr[:, 1] / pvr[:, 0]

In [ ]:
plt.figure(figsize=(10, 6))
fig = plt.scatter(pvr[:, 0], pvr[:, 1],
                  c=psr, cmap='coolwarm')
cb = plt.colorbar(fig)
cb.set_label('Sharpe ratio')
plt.xlabel('expected volatility')
plt.ylabel('expected return')
plt.title(' | '.join(symbols));

## Performance Out-of-Sample

In [ ]:
bnds = len(symbols) * [(0, 1),]
bnds

In [ ]:
cons = {'type': 'eq', 'fun': lambda weights: weights.sum() - 1}

In [ ]:
opt_weights = {}
for year in range(2010, 2019):
    rets_ = rets[symbols].loc[f'{year - 1}-01-01':f'{year}-12-31']
    ow = minimize(lambda weights: -port_sharpe(rets_, weights),
                  len(symbols) * [1 / len(symbols)],
                  bounds=bnds,
                  constraints=cons)['x']
    opt_weights[year] = ow

In [ ]:
opt_weights

In [ ]:
res = pd.DataFrame()
for year in range(2010, 2019):
    rets_ = rets[symbols].loc[f'{year}-01-01':f'{year}-12-31']
    epv = port_volatility(rets_, opt_weights[year])
    epr = port_return(rets_, opt_weights[year])
    esr = epr / epv
    rets_ = rets[symbols].loc[f'{year + 1}-01-01':f'{year + 1}-12-31']
    rpv = port_volatility(rets_, opt_weights[year])
    rpr = port_return(rets_, opt_weights[year])
    rsr = rpr / rpv
    res = pd.concat([res, pd.DataFrame({'epv': epv, 'epr': epr, 'esr': esr,
                                   'rpv': rpv, 'rpr': rpr, 'rsr': rsr},
                                  index=[year + 1])])

In [ ]:
res

In [ ]:
res.mean()

In [ ]:
(res[['rpv', 'rpr', 'rsr']].mean().values -
 res[['epv', 'epr', 'esr']].mean().values)

In [ ]:
res[['epv', 'rpv']].corr()

In [ ]:
res[['epv', 'rpv']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Portfolio Volatility');

In [ ]:
res[['epr', 'rpr']].corr()

In [ ]:
res[['epr', 'rpr']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Portfolio Return');

In [ ]:
res[['esr', 'rsr']].corr()

In [ ]:
res[['esr', 'rsr']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Sharpe Ratio');

## Historical Statistics over Time

In [ ]:
symbol = 'AAPL.O'

In [ ]:
data = pd.DataFrame(raw[symbol])

In [ ]:
data['r'] = np.log(data / data.shift(1))

In [ ]:
data['mu'] = data['r'].rolling(252).mean()

In [ ]:
data['std'] = data['r'].rolling(252).std()

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.head()

In [ ]:
(data['mu'] * 252).plot(figsize=(10, 6));  # annualized

In [ ]:
(data['std'] * 252 ** 0.5).plot(figsize=(10, 6));  # annualized

In [ ]:
data[['mu', 'std']].plot(figsize=(10, 6));

In [ ]:
(data[['mu', 'std']] / data[['mu', 'std']].iloc[0]).plot(figsize=(10, 6));

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>